In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Read in stats and weather (which stats?  cumulative row for each player and a weekly row for each player?)

In [3]:
# Read in Stats
rushing = pd.read_csv('./nfl_data/weekly/cbs_weekly/rushing_through_week-23.csv')
receiving = pd.read_csv('./nfl_data/weekly/cbs_weekly/receiving_through_week-23.csv')
passing = pd.read_csv('./nfl_data/weekly/cbs_weekly/passing_through_week-23.csv')

# Read in Weather
weather = pd.read_csv('./nfl_data/weekly/weather/all_weeks_weather.csv')

print(f'the shape of rushing is {rushing.shape}')
print(f'the shape of receiving is {receiving.shape}')
print(f'the shape of passing is {passing.shape}')

the shape of rushing is (106, 12)
the shape of receiving is (175, 12)
the shape of passing is (26, 13)


In [4]:
rushing.shape, receiving.shape, passing.shape

((106, 12), (175, 12), (26, 13))

In [5]:
rushing.head()

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos
0,Devin Singletary,BUF 47 - NE 17,21,16,81,2,3,13,4,0,19,RB
1,Jerick McKinnon,KC 42 - PIT 21,20,12,61,0,6,81,6,1,19,RB
2,Elijah Mitchell,SF 23 - DAL 17,15,27,96,1,1,-11,2,0,19,RB
3,Giovani Bernard,TB 31 - PHI 15,13,13,44,1,5,39,7,0,19,RB
4,Josh Jacobs,CIN 26 - LV 19,12,13,83,0,4,44,5,0,19,RB


In [6]:
r_r = pd.merge(rushing, receiving, how = 'outer')
stats = pd.merge(r_r,passing, how = 'outer')

In [7]:
stats.shape

(307, 17)

In [8]:
stats.columns

Index(['Player', 'Game', 'FPTS  Fantasy Points', 'ATT  Rushing Attempts',
       'YDS  Rushing Yards', 'TD  Rushing Touchdowns', 'REC  Receptions',
       'YDS  Receiving Yards', 'TGT  Targets', 'TD  Receiving Touchdowns',
       'Week', 'Pos', 'CMP  Pass Completions', 'ATT  Pass Attempts',
       'YDS  Passing Yards', 'TD  Touchdown Passes',
       'INT  Interceptions Thrown'],
      dtype='object')

In [9]:
stats.fillna(0, inplace = True)
stats.head()

,Player,Game,FPTS Fantasy Points,ATT Rushing Attempts,YDS Rushing Yards,TD Rushing Touchdowns,REC Receptions,YDS Receiving Yards,TGT Targets,TD Receiving Touchdowns,Week,Pos,CMP Pass Completions,ATT Pass Attempts,YDS Passing Yards,TD Touchdown Passes,INT Interceptions Thrown
0,Devin Singletary,BUF 47 - NE 17,21,16,81,2,3.0,13.0,4.0,0.0,19,RB,0.0,0.0,0.0,0.0,0.0
1,Devin Singletary,BUF 47 - NE 17,21,16,81,2,3.0,13.0,4.0,0.0,19,RB,0.0,0.0,0.0,0.0,0.0
2,Jerick McKinnon,KC 42 - PIT 21,20,12,61,0,6.0,81.0,6.0,1.0,19,RB,0.0,0.0,0.0,0.0,0.0
3,Jerick McKinnon,KC 42 - PIT 21,20,12,61,0,6.0,81.0,6.0,1.0,19,RB,0.0,0.0,0.0,0.0,0.0
4,Elijah Mitchell,SF 23 - DAL 17,15,27,96,1,1.0,-11.0,2.0,0.0,19,RB,0.0,0.0,0.0,0.0,0.0


In [10]:
weather.head()

,Away,Home,Forecast,Extended Forecast,Wind,Week,Wind_Speed_MPH,Wind_Direction,Temp,Weather_Desc
0,Cowboys,Buccaneers,80f Humid and Partly Cloudy,Humid and Partly Cloudy. Rain until evening.,6m WSW,1,6,WSW,80,Humid and Partly Cloudy
1,Eagles,Falcons,DOME,Clear. Clear throughout the day.,0,1,0,0,0,0
2,Steelers,Bills,72f Mostly Cloudy,Mostly Cloudy. Rain overnight.,12m WSW,1,12,WSW,72,Mostly Cloudy
3,Vikings,Bengals,82f Clear,Clear. Clear throughout the day.,11m SW,1,11,SW,82,Clear
4,49ers,Lions,DOME,Mostly Cloudy. Mostly cloudy throughout the day.,0,1,0,0,0,0


### Merge stats and weather

In [11]:
df_merged = pd.merge(stats, weather, left_on = ['Mascot', 'Week'], right_on = ['Home', 'Week'], how = 'inner')

KeyError: 'Mascot'